In [59]:
from openai import OpenAI
import os

# Typhoon API

In [60]:
OPENTYPHOON_API_KEY = 'sk-T6zybQdRNpKgcq8Ri3oYO3vrToJNsgLPNz5ekx02UF0jtWkN'
client = OpenAI(
    api_key=OPENTYPHOON_API_KEY,
    base_url="https://api.opentyphoon.ai/v1",
)

# Chat example

In [61]:
%%time
stream = client.chat.completions.create(
    model="typhoon-v1.5-instruct",
    messages=[
        {
            "role": "system",
            "content": "You are the best data scientist in inventory mangaement"
        },
        {
            "role": "user",
            "content": "Give me an example of inventory management"
        }
    ],
    max_tokens=300,
    temperature=0.8,
    top_p=1,
    stream=False,
)

CPU times: user 6.39 ms, sys: 1.78 ms, total: 8.17 ms
Wall time: 1.62 s


In [62]:
stream.choices[0].message.content

"Sure, here's an example: Consider a retail store that sells electronics. The store needs to manage its inventory of smartphones. The inventory management system should track the number of smartphones available in different colors and storage capacities. When a customer purchases a smartphone, the system should automatically update the inventory numbers. Additionally, the system should provide alerts when the stock levels of specific smartphones fall below certain thresholds, so the store can reorder before running out of stock."

# LangChain

In [63]:
from langchain import OpenAI, LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

In [64]:
OPENTYPHOON_API_KEY = 'sk-T6zybQdRNpKgcq8Ri3oYO3vrToJNsgLPNz5ekx02UF0jtWkN'
base_url = "https://api.opentyphoon.ai/v1"

llm = ChatOpenAI(
    model="typhoon-v1.5-instruct",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key= OPENTYPHOON_API_KEY,
    base_url= base_url,
)

In [65]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x12ff598d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12ff68950>, model_name='typhoon-v1.5-instruct', temperature=0.0, openai_api_key=SecretStr('**********'), openai_api_base='https://api.opentyphoon.ai/v1', openai_proxy='')

In [66]:
messages = [
    SystemMessage(content="Translate the following from English into Thai"),
    HumanMessage(content="dog!"),
]


In [67]:
llm.invoke(messages).content

'สุนัข!'

# langchan Table Agent

In [68]:
import pandas as pd
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [69]:
class pandas_code_Agent():
    def __init__(self,dataframe,prefix_prompt,input_prompt) -> None:
        self.llm = ChatOpenAI(
            model="typhoon-v1.5-instruct",
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2,
            api_key= OPENTYPHOON_API_KEY,
            base_url= base_url,
            )
        self.dataframe = dataframe
        self.prefix_prompt = prefix_prompt
        self.input_prompt =input_prompt
        self.agent = create_pandas_dataframe_agent(
            self.llm,
            self.dataframe,
            verbose = True,
            agent_type = AgentType.OPENAI_FUNCTIONS,
            prefix = prefix_prompt,
            allow_dangerous_code=True
            )
    
    def create_code(self):
        return self.agent.invoke(self.input_prompt)

In [70]:
df = pd.read_csv('./stock_full.csv')

In [71]:
pandas_agent = pandas_code_Agent(dataframe=df,prefix_prompt="you are the best data sciantist, that give output ONLY A CODE",
                               input_prompt="Do feature extraction from time in this data frame")

In [72]:
answer = pandas_agent.create_code()



> Entering new AgentExecutor chain...
To perform feature extraction on the time-related features in the given DataFrame, you can use the following Python code:

```python
import pandas as pd
from datetime import datetime

# Assuming 'df' is your DataFrame
df = pd.DataFrame({'day': [17, 10, 31, 29, 30],
                   'onth': [6, 8, 7, 9, nan],
                   'target': [0, 1.16162, 0, 0, 97.4578]})

# Extracting year and month names
df['year'] = df['month'].apply(lambda x: datetime(2000+x, 1, 1).year) \
   .fillna(method='ffill').astype(int)
df['month_name'] = df['month'].apply(lambda x: datetime(2000+x, 1, 1).strftime('%B')) \
   .fillna(method='ffill')

# Printing the updated DataFrame
print(df)
```

This code will create a new DataFrame with additional columns for the extracted features: 'year' and 'onth_name'. The 'year' column is created by filling the missing values with the previous non-missing value's year. The 'onth_name' column is created by converting the month numb

In [73]:
answer

{'input': 'Do feature extraction from time in this data frame',
 'output': "To perform feature extraction on the time-related features in the given DataFrame, you can use the following Python code:\n\n```python\nimport pandas as pd\nfrom datetime import datetime\n\n# Assuming 'df' is your DataFrame\ndf = pd.DataFrame({'day': [17, 10, 31, 29, 30],\n                   'onth': [6, 8, 7, 9, nan],\n                   'target': [0, 1.16162, 0, 0, 97.4578]})\n\n# Extracting year and month names\ndf['year'] = df['month'].apply(lambda x: datetime(2000+x, 1, 1).year) \\\n   .fillna(method='ffill').astype(int)\ndf['month_name'] = df['month'].apply(lambda x: datetime(2000+x, 1, 1).strftime('%B')) \\\n   .fillna(method='ffill')\n\n# Printing the updated DataFrame\nprint(df)\n```\n\nThis code will create a new DataFrame with additional columns for the extracted features: 'year' and 'onth_name'. The 'year' column is created by filling the missing values with the previous non-missing value's year. The

In [74]:
answer['output']

"To perform feature extraction on the time-related features in the given DataFrame, you can use the following Python code:\n\n```python\nimport pandas as pd\nfrom datetime import datetime\n\n# Assuming 'df' is your DataFrame\ndf = pd.DataFrame({'day': [17, 10, 31, 29, 30],\n                   'onth': [6, 8, 7, 9, nan],\n                   'target': [0, 1.16162, 0, 0, 97.4578]})\n\n# Extracting year and month names\ndf['year'] = df['month'].apply(lambda x: datetime(2000+x, 1, 1).year) \\\n   .fillna(method='ffill').astype(int)\ndf['month_name'] = df['month'].apply(lambda x: datetime(2000+x, 1, 1).strftime('%B')) \\\n   .fillna(method='ffill')\n\n# Printing the updated DataFrame\nprint(df)\n```\n\nThis code will create a new DataFrame with additional columns for the extracted features: 'year' and 'onth_name'. The 'year' column is created by filling the missing values with the previous non-missing value's year. The 'onth_name' column is created by converting the month number to a string r

# langchain Agent

In [75]:
class normal_Agent():
    def __init__(self,system_prmopt) -> None:
        self.llm = ChatOpenAI(
            model="typhoon-v1.5-instruct",
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2,
            api_key= OPENTYPHOON_API_KEY,
            base_url= base_url,
            )
        self.system_prmopt = system_prmopt
        
    def create_code(self,user_prmopt): 
         return self.llm.invoke([
            SystemMessage(content=self.system_prmopt),
            HumanMessage(content=user_prmopt),
            ]).content
    
    

In [76]:
select_agent = normal_Agent("""From the text that I given. You have to give me only a feature extraction code.
                            Input:
                            The feature extraction code provided is written in Python using the Pandas library. It extracts two time-related features from the given DataFrame: 'year' and 'month_name'. Here's the code snippet again for reference:

                            ```python
                            import pandas as pd
                            from datetime import datetime

                            # Assuming 'df' is your DataFrame
                            df = pd.DataFrame({'day': [17, 10, 31, 29, 30],
                                               'onth': [6, 8, 7, 9, nan],
                                               'target': [0, 1.16162, 0, 0, 97.4578]})

                            # Extracting
                            df['Weight (kg.)/Length (m.)'] = df['Weight (kg.)']/df['Length (m.)']
                            df['thickness_mean'] df['thickness'].mean(

                            # Printing the updated DataFrame
                            print(df)
                            ```
                            
                            Output:
                            df['Weight (kg.)/Length (m.)'] = df['Weight (kg.)']/df['Length (m.)']
                            df['thickness_mean'] df['thickness'].mean()""")

In [77]:
print(select_agent.create_code(answer['output']))

Thank you for providing the feature extraction code. Based on the code you provided, I will now extract the relevant lines of code that perform the feature extraction.

Here are the lines of code that extract the 'year' and 'month_name' features from the given DataFrame:

```python
df['year'] = df['month'].apply(lambda x: datetime(2000+x, 1, 1).year) \
   .fillna(method='ffill').astype(int)
df['month_name'] = df['month'].apply(lambda x: datetime(2000+x, 1, 1).strftime('%B')) \
   .fillna(method='ffill')
```

These lines of code will create a new DataFrame with additional columns for the extracted features: 'year' and 'month_name'. The 'year' column is created by filling the missing values with the previous non-missing value's year. The 'month_name' column is created by converting the month number to a string representation of the month name, filling the missing values with the previous non-missing value's month name.


# All pipeline

In [78]:
# read data frame
df = pd.read_csv('./stock_full.csv')

In [79]:
# Create Data sciantist Agent
pandas_agent_1 = pandas_code_Agent(dataframe=df,prefix_prompt="you are the best data sciantist, that expert in Machine Learning",
                               input_prompt="Do feature extraction from time in this data frame")
pandas_agent_2 = pandas_code_Agent(dataframe=df,prefix_prompt="you are the best data sciantist, that give output ONLY A CODE",
                               input_prompt="Do feature extraction from time in this data frame")
pandas_agent_3 = pandas_code_Agent(dataframe=df,prefix_prompt="you are the best data sciantist, that give output ONLY A CODE",
                               input_prompt="Do feature extraction from time in this data frame")


# Feature select Agent
select_feature_agent = normal_Agent("you are Machine learning Engineer")